# Capstone Project 2

## Building the dataset

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
import re
import time
from tqdm import tqdm

### Building a dataframe with the 30 most rated boardgames

In [2]:
def req(msg, slp=0.2):
    """Make fault tolerant BGG server requests."""
    # Sleep to make sure you are not pinging the server to frequently
    sleep(slp)
    # Keep trying requests until status-code is 200
    status_code = 500
    while status_code != 200:
        sleep(slp)
        try:
            r = requests.get(msg)
            status_code = r.status_code
            # if status_code != 200:
                # print("Server Error! Response Code %i. Retrying..." % (r.status_code))
        except:
            # print("An exception has occurred, probably a momentory loss of connection. Waiting three seconds...")
            sleep(3)
    return r

In [3]:
def request(msg, slp=1):
    '''A wrapper to make robust https requests.'''
    status_code = 500  # Want to get a status-code of 200
    while status_code != 200:
        sleep(slp)  # Don't ping the server too often
        try:
            r = requests.get(msg)
            status_code = r.status_code
            if status_code != 200:
                print("Server Error! Response Code %i. Retrying..." % (r.status_code))
        except:
            print("An exception has occurred, probably a momentory loss of connection. Waiting one seconds...")
            sleep(1)
    return r

In [4]:
# Get full HTML for a specific page in the full listing of boardgames sorted by 
r = request("https://boardgamegeek.com/browse/boardgame/page/%i?sort=numvoters&sortdir=desc" % (1,))
soup = BeautifulSoup(r.text, "html.parser")    
    
# Get rows for the table listing all the games on this page
table = soup.find_all("tr", attrs={"id": "row_"})  # Get list of all the rows (tags) in the list of games on this page
df = pd.DataFrame(columns=["id", "name", "nrate", "pic_url"], index=range(len(table)))  # DF to hold this pages results
    
# Loop through each row and pull out the info for that game
for idx, row in enumerate(table):
    # Row may or may not start with a "boardgame rank" link, if YES then strip it
    links = row.find_all("a")
    if "name" in links[0].attrs.keys():
        del links[0]
    gamelink = links[1]  # Get the relative URL for the specific game
    gameid = int(gamelink["href"].split("/")[2])  # Get the game ID by parsing the relative URL
    gamename = gamelink.contents[0]  # Get the actual name of the game as the link contents
    imlink = links[0]  # Get the URL for the game thumbnail
    thumbnail = imlink.contents[0]["src"]

    ratings_str = row.find_all("td", attrs={"class": "collection_bggrating"})[2].contents[0]
    nratings = int("".join(ratings_str.split()))

    df.iloc[idx, :] = [gameid, gamename, nratings,thumbnail]

sleep(2) # Keep the BGG server happy.

# Prepare a "# of FULL pages of ratings" column to track # API calls needed
df["nfullpages"] = (df["nrate"]-50).apply(round, ndigits=-2)/100  # Round DOWN to nearest 100

# 30 most nrated game
df=df.iloc[0:30]
del df['pic_url']

In [5]:
print(df.shape)
df

(30, 4)


,id,name,nrate,nfullpages
0,13,Catan,87248,872.0
1,822,Carcassonne,86933,869.0
2,30549,Pandemic,85707,857.0
3,68448,7 Wonders,70971,709.0
4,36218,Dominion,69518,695.0
5,9209,Ticket to Ride,61798,617.0
6,31260,Agricola,57976,579.0
7,3076,Puerto Rico,57362,573.0
8,40692,Small World,54626,546.0
9,178900,Codenames,53304,533.0


### Building a dataframe with the reviews, rates and some information on the 30 most rated boardgames

In [6]:
def game_info (soup,ID):
    """Get the boardgame information"""
    name=soup('name')[0]["value"]
    year=soup('yearpublished')[0]["value"]
    min_play=soup('minplayers')[0]["value"]
    max_play=soup('maxplayers')[0]["value"]
    min_time=soup('minplaytime')[0]["value"]
    max_time=soup('maxplaytime')[0]["value"]
    min_age=soup('minage')[0]["value"]

    Category=[]
    for a in range(0,10):
        if soup('link')[a]['type']=="boardgamecategory":   
            Category.append(soup('link')[a]["value"])
    category=Category[0]
    return name,year,min_play,max_play,min_time,max_time,min_age,category

In [7]:
def game_rating_reviews (soup,review,username,rating):
    """Get the boardgames rates and reviews"""
    for nb in range(0,99):
        review.append(soup('comment')[nb]["value"])
        username.append(soup('comment')[nb]["username"])
        rate.append(soup('comment')[nb]["rating"])
    return review,username,rate

In [116]:
# transform the column with the id as a object column and the column with the number of pages as an integer column
df.id=df.id.astype(str)
df.nfullpages=df.nfullpages.astype(int)
# add a progression bar
for i in tqdm(range(20)):
    # Go through the 30 boardgames
    for a in range(0,1):############################################### 30
        nb_rate=df.nrate[a]
        ID=df.id[a]
        nb_page=df.nfullpages[a]
        # Go through the n pages of reviews
        for b in range (0,nb_page):
            b_str=str(b)
            # Access to each of the boardgame information page by page
            url="https://www.boardgamegeek.com/xmlapi2/thing?id="+ID+"&page="+b_str+"&ratingcomments=1"
            r=requests.get(url)
            soup=BeautifulSoup(r.text)
            # Check if the connection works
            if r.status_code==200:
                review=[]
                username=[]
                rate=[]
                # Get the boardgames information
                name,year,min_play,max_play,min_time,max_time,min_age,category=game_info(soup,ID)
                # Get the boardgames rates & reviews
                review,username,rate=game_rating_reviews(soup,review,username,rate)
                # Building the dataframe for each boardgame
                data={'id':ID,'name':name,'year':year,'min_play':min_play,'max_play':max_play,'min_time':min_time,
                     'max_time':max_time,'min_age':min_age,'category':category,'nb_rate':nb_rate,'username':username,
                     'review':review,'rate':rate}
                df_bgg_onegame=pd.DataFrame(data)
                # Drop the raws without any review
                df_bgg_onegame=df_bgg_onegame.drop(df_bgg_onegame[df_bgg_onegame.review==''].index) 
                # Concatenate the dataframes
                if a==0 & b=='0':
                    df_bgg=df_bgg_onegame.copy()
                else:
                    df_bgg=df_bgg.append(df_bgg_onegame).reset_index(drop=True)
    time.sleep(3)





  0%|                                                                                           | 0/20 [00:00<?, ?it/s]



  5%|████                                                                            | 1/20 [05:27<1:43:49, 327.89s/it]



 10%|████████                                                                        | 2/20 [09:54<1:32:51, 309.53s/it]



 15%|████████████                                                                    | 3/20 [14:20<1:24:00, 296.49s/it]



 20%|████████████████                                                                | 4/20 [19:39<1:20:50, 303.17s/it]



 25%|████████████████████                                                            | 5/20 [26:08<1:22:15, 329.01s/it]



 30%|████████████████████████                                                        | 6/20 [33:27<1:24:28, 362.01s/it]



 35%|████████████████████████████                                                    | 7/20 [42:00<1:28:12, 407.10s/it]



 40%|███████

In [21]:
# transform the column with the id as a object column and the column with the number of pages as an integer column
df.id=df.id.astype(str)
df.nfullpages=df.nfullpages.astype(int)
# add a progression bar
for i in tqdm(range(20)):
    # Number of boardgame
    a=4################################################################################################??????????
    nb_rate=df.nrate[a]
    ID=df.id[a]
    nb_page=df.nfullpages[a]
    # Go through the n pages of reviews
    for b in range (0,nb_page):
        b_str=str(b)
        # Access to each of the boardgame information page by page
        url="https://www.boardgamegeek.com/xmlapi2/thing?id="+ID+"&page="+b_str+"&ratingcomments=1"
        r=requests.get(url)
        soup=BeautifulSoup(r.text)
        # Check if the connection works
        if r.status_code==200:
            review=[]
            username=[]
            rate=[]
            # Get the boardgames information
            name,year,min_play,max_play,min_time,max_time,min_age,category=game_info(soup,ID)
            # Get the boardgames rates & reviews
            review,username,rate=game_rating_reviews(soup,review,username,rate)
            # Building the dataframe for each boardgame
            data={'id':ID,'name':name,'year':year,'min_play':min_play,'max_play':max_play,'min_time':min_time,
                'max_time':max_time,'min_age':min_age,'category':category,'nb_rate':nb_rate,'username':username,
                'review':review,'rate':rate}
            df_bgg_onegame=pd.DataFrame(data)
            # Drop the raws without any review
            df_bgg_onegame=df_bgg_onegame.drop(df_bgg_onegame[df_bgg_onegame.review==''].index) 
            # Concatenate the dataframes
            if b==0:
                df_bgg=df_bgg_onegame.copy()
            else:
                df_bgg=df_bgg.append(df_bgg_onegame).reset_index(drop=True)
    time.sleep(3)

100%|███████████████████████████████████████████████████████████████████████████████| 20/20 [1:07:21<00:00, 217.07s/it]


In [22]:
print(df_bgg.shape)
df_bgg

(9240, 13)


,id,name,year,min_play,max_play,min_time,max_time,min_age,category,nb_rate,username,review,rate
0,36218,Dominion,2008,2,4,30,30,13,Card Game,69518,shaveandahaircut,[138 plays] Boy this is fun! If I get sick of ...,10
1,36218,Dominion,2008,2,4,30,30,13,Card Game,69518,statonv,Full disclosure--I am a part of the developmen...,10
2,36218,Dominion,2008,2,4,30,30,13,Card Game,69518,jeffwiles,I feel like we (personally) haven't even begun...,10
3,36218,Dominion,2008,2,4,30,30,13,Card Game,69518,Cashtool,"Prototype play. \nEasy to learn, with great re...",10
4,36218,Dominion,2008,2,4,30,30,13,Card Game,69518,the_spy,Simply a great game. Won't ever decline a game.,10
5,36218,Dominion,2008,2,4,30,30,13,Card Game,69518,JanMS,(2-4) - 30 Min.,10
6,36218,Dominion,2008,2,4,30,30,13,Card Game,69518,manhattandoctor,+ Alchemy\n+ Cornucopia\n+ Dark Ages\n+ Guilds...,10
7,36218,Dominion,2008,2,4,30,30,13,Card Game,69518,locusshifter,It's a really great game that I suck at. I pre...,10
8,36218,Dominion,2008,2,4,30,30,13,Card Game,69518,Ron 100mark,With expansions this game gets top ranking. Ea...,10
9,36218,Dominion,2008,2,4,30,30,13,Card Game,69518,jtakagi,The unbelievable amount of hype/buzz for this ...,10


In [129]:
# Boardgame #0
#df_bgg_0=df_bgg.copy()
# Register the final dataframe as csv format
df_bgg_0.to_csv("bgg_review_0.csv", index=False, encoding="utf-8")
print(df_bgg_0.shape)
df_bgg_0.head()

(14210, 13)


,id,name,year,min_play,max_play,min_time,max_time,min_age,category,nb_rate,username,review,rate
0,13,Catan,1995,3,4,60,120,10,Negotiation,87205,Deleted010518,2008-04-05 \n\nDave Loved the game and pushed ...,7.2
1,13,Catan,1995,3,4,60,120,10,Negotiation,87205,fliptrackr,Like this game (even though I don't own it). ...,7.2
2,13,Catan,1995,3,4,60,120,10,Negotiation,87205,ShirKhan,"Classic. Besides that, fairly random and often...",7.2
3,13,Catan,1995,3,4,60,120,10,Negotiation,87205,bkunes,"Enjoyable game, challenge is getting at least ...",7.2
4,13,Catan,1995,3,4,60,120,10,Negotiation,87205,BenjoBaker,What can I say? If you haven't played Settler...,7.2


In [14]:
# Boardgame #1
df_bgg_1=df_bgg_0.append(df_bgg).reset_index(drop=True)
df_bgg_1.to_csv("bgg_review_1.csv", index=False, encoding="utf-8")
print(df_bgg_1.shape)
df_bgg_1.head()

(26527, 13)


,id,name,year,min_play,max_play,min_time,max_time,min_age,category,nb_rate,username,review,rate
0,13,Catan,1995,3,4,60,120,10,Negotiation,87205,Deleted010518,2008-04-05 \r\n\r\nDave Loved the game and pus...,7.2
1,13,Catan,1995,3,4,60,120,10,Negotiation,87205,fliptrackr,Like this game (even though I don't own it). ...,7.2
2,13,Catan,1995,3,4,60,120,10,Negotiation,87205,ShirKhan,"Classic. Besides that, fairly random and often...",7.2
3,13,Catan,1995,3,4,60,120,10,Negotiation,87205,bkunes,"Enjoyable game, challenge is getting at least ...",7.2
4,13,Catan,1995,3,4,60,120,10,Negotiation,87205,BenjoBaker,What can I say? If you haven't played Settler...,7.2


In [17]:
# Boardgame #2
df_bgg_2=df_bgg_1.append(df_bgg).reset_index(drop=True)
df_bgg_2.to_csv("bgg_review_2.csv", index=False, encoding="utf-8")
print(df_bgg_2.shape)
df_bgg_2.head()

(35768, 13)


,id,name,year,min_play,max_play,min_time,max_time,min_age,category,nb_rate,username,review,rate
0,13,Catan,1995,3,4,60,120,10,Negotiation,87205,Deleted010518,2008-04-05 \r\n\r\nDave Loved the game and pus...,7.2
1,13,Catan,1995,3,4,60,120,10,Negotiation,87205,fliptrackr,Like this game (even though I don't own it). ...,7.2
2,13,Catan,1995,3,4,60,120,10,Negotiation,87205,ShirKhan,"Classic. Besides that, fairly random and often...",7.2
3,13,Catan,1995,3,4,60,120,10,Negotiation,87205,bkunes,"Enjoyable game, challenge is getting at least ...",7.2
4,13,Catan,1995,3,4,60,120,10,Negotiation,87205,BenjoBaker,What can I say? If you haven't played Settler...,7.2


In [20]:
# Boardgame #3
df_bgg_3=df_bgg_2.append(df_bgg).reset_index(drop=True)
df_bgg_3.to_csv("bgg_review_3.csv", index=False, encoding="utf-8")
print(df_bgg_3.shape)
df_bgg_3.head()

(44366, 13)


,id,name,year,min_play,max_play,min_time,max_time,min_age,category,nb_rate,username,review,rate
0,13,Catan,1995,3,4,60,120,10,Negotiation,87205,Deleted010518,2008-04-05 \r\n\r\nDave Loved the game and pus...,7.2
1,13,Catan,1995,3,4,60,120,10,Negotiation,87205,fliptrackr,Like this game (even though I don't own it). ...,7.2
2,13,Catan,1995,3,4,60,120,10,Negotiation,87205,ShirKhan,"Classic. Besides that, fairly random and often...",7.2
3,13,Catan,1995,3,4,60,120,10,Negotiation,87205,bkunes,"Enjoyable game, challenge is getting at least ...",7.2
4,13,Catan,1995,3,4,60,120,10,Negotiation,87205,BenjoBaker,What can I say? If you haven't played Settler...,7.2


In [23]:
# Boardgame #4
df_bgg_4=df_bgg_3.append(df_bgg).reset_index(drop=True)
df_bgg_4.to_csv("bgg_review_4.csv", index=False, encoding="utf-8")
print(df_bgg_4.shape)
df_bgg_4.head()

(53606, 13)


,id,name,year,min_play,max_play,min_time,max_time,min_age,category,nb_rate,username,review,rate
0,13,Catan,1995,3,4,60,120,10,Negotiation,87205,Deleted010518,2008-04-05 \r\n\r\nDave Loved the game and pus...,7.2
1,13,Catan,1995,3,4,60,120,10,Negotiation,87205,fliptrackr,Like this game (even though I don't own it). ...,7.2
2,13,Catan,1995,3,4,60,120,10,Negotiation,87205,ShirKhan,"Classic. Besides that, fairly random and often...",7.2
3,13,Catan,1995,3,4,60,120,10,Negotiation,87205,bkunes,"Enjoyable game, challenge is getting at least ...",7.2
4,13,Catan,1995,3,4,60,120,10,Negotiation,87205,BenjoBaker,What can I say? If you haven't played Settler...,7.2


In [ ]:
# Boardgame #5
df_bgg_5=df_bgg_4.append(df_bgg).reset_index(drop=True)
df_bgg_5.to_csv("bgg_review_5.csv", index=False, encoding="utf-8")
print(df_bgg_5.shape)
df_bgg_5.head()

In [ ]:
# Boardgame #6
df_bgg_6=df_bgg_5.append(df_bgg).reset_index(drop=True)
df_bgg_6.to_csv("bgg_review_6.csv", index=False, encoding="utf-8")
print(df_bgg_6.shape)
df_bgg_6.head()

In [ ]:
# Boardgame #7
df_bgg_7=df_bgg_6.append(df_bgg).reset_index(drop=True)
df_bgg_7.to_csv("bgg_review_7.csv", index=False, encoding="utf-8")
print(df_bgg_7.shape)
df_bgg_7.head()

In [ ]:
# Boardgame #8
df_bgg_8=df_bgg_7.append(df_bgg).reset_index(drop=True)
df_bgg_8.to_csv("bgg_review_8.csv", index=False, encoding="utf-8")
print(df_bgg_8.shape)
df_bgg_8.head()

In [ ]:
# Boardgame #9
df_bgg_9=df_bgg_8.append(df_bgg).reset_index(drop=True)
df_bgg_9.to_csv("bgg_review_9.csv", index=False, encoding="utf-8")
print(df_bgg_9.shape)
df_bgg_9.head()

In [ ]:
# Boardgame #10
df_bgg_10=df_bgg_9.append(df_bgg).reset_index(drop=True)
df_bgg_10.to_csv("bgg_review_10.csv", index=False, encoding="utf-8")
print(df_bgg_10.shape)
df_bgg_10.head()

In [ ]:
# Boardgame #11
df_bgg_11=df_bgg_10.append(df_bgg).reset_index(drop=True)
df_bgg_11.to_csv("bgg_review_11.csv", index=False, encoding="utf-8")
print(df_bgg_11.shape)
df_bgg_11.head()

In [ ]:
# Boardgame #12
df_bgg_12=df_bgg_11.append(df_bgg).reset_index(drop=True)
df_bgg_12.to_csv("bgg_review_12.csv", index=False, encoding="utf-8")
print(df_bgg_12.shape)
df_bgg_12.head()

In [ ]:
# Boardgame #13
df_bgg_13=df_bgg_12.append(df_bgg).reset_index(drop=True)
df_bgg_13.to_csv("bgg_review_13.csv", index=False, encoding="utf-8")
print(df_bgg_13.shape)
df_bgg_13.head()

In [ ]:
# Boardgame #14
df_bgg_14=df_bgg_13.append(df_bgg).reset_index(drop=True)
df_bgg_14.to_csv("bgg_review_14.csv", index=False, encoding="utf-8")
print(df_bgg_14.shape)
df_bgg_14.head()

In [ ]:
# Boardgame #15
df_bgg_15=df_bgg_14.append(df_bgg).reset_index(drop=True)
df_bgg_15.to_csv("bgg_review_15.csv", index=False, encoding="utf-8")
print(df_bgg_15.shape)
df_bgg_15.head()

In [ ]:
# Boardgame #16
df_bgg_16=df_bgg_15.append(df_bgg).reset_index(drop=True)
df_bgg_16.to_csv("bgg_review_6.csv", index=False, encoding="utf-8")
print(df_bgg_16.shape)
df_bgg_16.head()

In [ ]:
# Boardgame #17
df_bgg_17=df_bgg_16.append(df_bgg).reset_index(drop=True)
df_bgg_17.to_csv("bgg_review_17.csv", index=False, encoding="utf-8")
print(df_bgg_17.shape)
df_bgg_17.head()

In [ ]:
# Boardgame #18
df_bgg_18=df_bgg_17.append(df_bgg).reset_index(drop=True)
df_bgg_18.to_csv("bgg_review_18.csv", index=False, encoding="utf-8")
print(df_bgg_18.shape)
df_bgg_18.head()

In [ ]:
# Boardgame #19
df_bgg_19=df_bgg_18.append(df_bgg).reset_index(drop=True)
df_bgg_19.to_csv("bgg_review_19.csv", index=False, encoding="utf-8")
print(df_bgg_19.shape)
df_bgg_19.head()

In [ ]:
# Boardgame #20
df_bgg_20=df_bgg_19.append(df_bgg).reset_index(drop=True)
df_bgg_20.to_csv("bgg_review_20.csv", index=False, encoding="utf-8")
print(df_bgg_20.shape)
df_bgg_20.head()

In [ ]:
# Boardgame #21
df_bgg_21=df_bgg_20.append(df_bgg).reset_index(drop=True)
df_bgg_21.to_csv("bgg_review_21.csv", index=False, encoding="utf-8")
print(df_bgg_21.shape)
df_bgg_21.head()

In [ ]:
# Boardgame #22
df_bgg_22=df_bgg_21.append(df_bgg).reset_index(drop=True)
df_bgg_22.to_csv("bgg_review_22.csv", index=False, encoding="utf-8")
print(df_bgg_22.shape)
df_bgg_22.head()

In [ ]:
# Boardgame #23
df_bgg_23=df_bgg_22.append(df_bgg).reset_index(drop=True)
df_bgg_23.to_csv("bgg_review_23.csv", index=False, encoding="utf-8")
print(df_bgg_23.shape)
df_bgg_23.head()

In [ ]:
# Boardgame #24
df_bgg_24=df_bgg_23.append(df_bgg).reset_index(drop=True)
df_bgg_24.to_csv("bgg_review_24.csv", index=False, encoding="utf-8")
print(df_bgg_24.shape)
df_bgg_24.head()

In [ ]:
# Boardgame #25
df_bgg_25=df_bgg_24.append(df_bgg).reset_index(drop=True)
df_bgg_25.to_csv("bgg_review_25.csv", index=False, encoding="utf-8")
print(df_bgg_25.shape)
df_bgg_25.head()

In [ ]:
# Boardgame #26
df_bgg_26=df_bgg_25.append(df_bgg).reset_index(drop=True)
df_bgg_26.to_csv("bgg_review_26.csv", index=False, encoding="utf-8")
print(df_bgg_26.shape)
df_bgg_26.head()

In [ ]:
# Boardgame #27
df_bgg_27=df_bgg_26.append(df_bgg).reset_index(drop=True)
df_bgg_27.to_csv("bgg_review_27.csv", index=False, encoding="utf-8")
print(df_bgg_27.shape)
df_bgg_27.head()

In [ ]:
# Boardgame #28
df_bgg_28=df_bgg_27.append(df_bgg).reset_index(drop=True)
df_bgg_28.to_csv("bgg_review_28.csv", index=False, encoding="utf-8")
print(df_bgg_28.shape)
df_bgg_28.head()

In [ ]:
# Boardgame #29
df_bgg_29=df_bgg_28.append(df_bgg).reset_index(drop=True)
df_bgg_29.to_csv("bgg_review_29.csv", index=False, encoding="utf-8")
print(df_bgg_29.shape)
df_bgg_29.head()